# 2-1 extract 4grams from smali code

- step1 smali 코드 수집
- step2 smali 코드에서 4gram을 추출하여 전체 리스트 및 갯수 확인
- step3 smali 코드에서 비번하게 나타나는 10000개의 리스트를 가지고 malware, benign의 비율 확인
- step4 10000개의 리스트 중에서 영향도 있는 리스트만 각출
- step5 각 apk 파일들의 4gram 수집 및 정리


## step1 smali 코드 수집

In [7]:
import os

dir_names = ["Challenge_andro_2nd_dataset"]
dir_list = dict()

i = 0
print( "start")
for dir_name in dir_names:
    dir_name = dir_name + "_apktools"
    if dir_name.startswith('Benign'):
        tag = 'Benign'
    else :
        tag = 'Malware'
        
    for root, dirs, files in os.walk(dir_name):
        for f in files:
            if f.endswith(".smali"):
                dir_list[root + '/' + f ] = tag
            i+=1
            if i%10000 == 0:
                print( "{}                  \r"i, end =' ')
                    
print('len', len(dir_list))

. 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000 69000 70000 71000 72000 73000 74000 75000 76000 77000 78000 79000 80000 81000 82000 83000 84000 85000 86000 87000 88000 89000 90000 91000 92000 93000 94000 95000 96000 97000 98000 99000 100000 101000 102000 103000 104000 105000 106000 107000 108000 109000 110000 111000 112000 113000 114000 115000 116000 117000 118000 119000 120000 121000 122000 123000 124000 125000 126000 127000 128000 129000 130000 131000 132000 133000 134000 135000 136000 137000 138000 139000 140000 141000 142000 143000 144000 145000 146000 147000 148000 149000 150000 151000 152000 153000 154000 155000 156000 157000 158000 

In [8]:
#dir_list.keys()[:10]
a = dir_list.keys()
a = list(a)
a[:10]

['Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/e.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/c.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/a.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/f.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/b.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/d.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/h.smali',
 'Challenge_andro_2nd_dataset_apktools/f8e57f2d43b79ab9a390aa046b7e88c120b81d4b237d57bda535d539aa334faf/smali/com/a/g.

## step2 smali 코드에서 4gram을 추출하여 전체 리스트 및 갯수 확인

In [13]:
import pandas as pd

df = pd.read_csv("bm_df_smali_4gram_X.csv").T.drop('Unnamed: 0')
smali4_list = list(df.index)
smali4_list[:10]

["('&', 'invoke', 'virtual', 'v')",
 "('*', 'end', 'annotation', 'end')",
 "('*', 'end', 'annotation', 'prologue')",
 "('*', 'iget', 'object', 'v')",
 "('[', 'invoke', 'virtual', 'v')",
 "('[b', 'array', 'length', 'v')",
 "('[b', 'const', 'v', 'x')",
 "('[b', 'end', 'local', 'v')",
 "('[b', 'fill', 'array', 'data')",
 "('[b', 'i', 'move', 'result')"]

## step5 각 apk 파일들의 4gram 수집 및 정리

apk_word_list

파일로부터 데이터를 불러와서 해시 값에 따라 counter 값 추가

In [22]:
import nltk
from multiprocessing import Pool
from collections import Counter
import numpy as np

apk_word_list = dict()

def file2strings(key):
    special_char = ['$',',','.',';','-','=','<','>',
                    '1','2','3','4','5','6','7','8','9','0',
                    '{','}','(',')',':','`', '#', "'",'/','"']
    
    with open(key) as f:
        data = f.read()#.encode('ascii', 'replace')
        #data = data.decode('utf-8', 'ignore')
        data = data.lower()
        
        for char in special_char:
            data = data.replace(char,' ')
        
        data = data.split()
        fourgrams = nltk.ngrams(data, 4)
                
        word_count = Counter()
        filename = key.split("/")[1] # 'Benign_samples_1st_1500_apktools/b54a9bd9669e2bca118a58c4e08bafad9bacfa5717364226140b9d9180a2b5e0/smali/com/appsbar/CBFerreterias140547/ActivityWMenu$FbLoginDialogListener.smali'
        
        for word in list(fourgrams):
            word_count[str(word)] += 1
                
        return filename, dict(word_count.most_common())

with Pool(14) as p:
    print ("start")
    i = 0
    count = len(dir_list)
    
    for filename, word_count in p.imap(file2strings, list(dir_list.keys())):
        apk_word_list[filename] = word_count
        
        i += 1    
        if i %100 == 0:
            print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')
     
    print("{}%                              \r".format(i/count*100),flush=True,sep=' ', end = '')

start


In [15]:
apk_word_df = pd.DataFrame(apk_word_list, index=smali4_list)
apk_word_df = apk_word_df.fillna(0).T
apk_word_df

,"('&', 'invoke', 'virtual', 'v')","('*', 'end', 'annotation', 'end')","('*', 'end', 'annotation', 'prologue')","('*', 'iget', 'object', 'v')","('[', 'invoke', 'virtual', 'v')","('[b', 'array', 'length', 'v')","('[b', 'const', 'v', 'x')","('[b', 'end', 'local', 'v')","('[b', 'fill', 'array', 'data')","('[b', 'i', 'move', 'result')",...,"('z', 'v', 'line', 'cond_')","('z', 'v', 'line', 'const')","('z', 'v', 'line', 'iget')","('z', 'v', 'line', 'invoke')","('z', 'v', 'line', 'return')","('z', 'v', 'locals', 'param')","('z', 'v', 'locals', 'prologue')","('z', 'v', 'return', 'void')","('z', 'v', 'try_end_', 'catch')","('z', 'z', 'move', 'result')"
000ef8ee5e09458686ca98ac8914eafdc0e5e937ae2bcac90ce591a4ad22d023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001652ebc24cc76dbe3fb27d80eb137fa8b60832958ace7617c6d0c6fdb6d18a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00d56ee88d368ea75997bce26cd5c90a2d0be8981d355c82acf053768bad43f0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00d7f6f13d72210c232707853f28ca2f0267ccab605e7366c92e336faf6bf6bc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00e5635e0af8a17aca2e22f3fcbd52314c0aa16012ec7af30a31a42aea1dcfe8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00f130be2d96cb994c0b7465d6392b30675a8a1186688ef053e1a0c0b0be25dc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
012bfafab8a84bdeb62a22af97ccafaec924cef3329680787dad16dbb2992ca1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0160c026ef59c9c0fe74cfb26082858ce4def298114ef904ad50b8d56a7f8b50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01718bfe393c5235cdb559eafe1bc3921238e063c087881fed2fe3db187c4b5e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0178c9d429db092f01bfd5c920be9b2f18791d3b69b4a12eaffdd6e59094bff0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
apk_word_df.sum()

('&', 'invoke', 'virtual', 'v')                0.0
('*', 'end', 'annotation', 'end')              0.0
('*', 'end', 'annotation', 'prologue')         0.0
('*', 'iget', 'object', 'v')                   0.0
('[', 'invoke', 'virtual', 'v')                0.0
('[b', 'array', 'length', 'v')                 0.0
('[b', 'const', 'v', 'x')                      0.0
('[b', 'end', 'local', 'v')                    0.0
('[b', 'fill', 'array', 'data')                0.0
('[b', 'i', 'move', 'result')                  0.0
('[b', 'iget', 'v', 'p')                       0.0
('[b', 'line', 'local', 'v')                   0.0
('[b', 'ljava', 'lang', 'string')              0.0
('[b', 'locals', 'param', 'p')                 0.0
('[b', 'move', 'result', 'object')             0.0
('[c', 'move', 'result', 'object')             0.0
('[f', 'aget', 'v', 'v')                       0.0
('[f', 'const', 'v', 'x')                      0.0
('[i', 'add', 'int', 'lit')                    0.0
('[i', 'aget', 'v', 'v')       

In [17]:
len(list(apk_word_df))

7361

In [18]:
apk_word_df.to_csv("2nd_smali_4gram.csv")

In [20]:
apk_word_df = pd.DataFrame(apk_word_list)

In [21]:
apk_word_df

,000ef8ee5e09458686ca98ac8914eafdc0e5e937ae2bcac90ce591a4ad22d023,001652ebc24cc76dbe3fb27d80eb137fa8b60832958ace7617c6d0c6fdb6d18a,00d56ee88d368ea75997bce26cd5c90a2d0be8981d355c82acf053768bad43f0,00d7f6f13d72210c232707853f28ca2f0267ccab605e7366c92e336faf6bf6bc,00e5635e0af8a17aca2e22f3fcbd52314c0aa16012ec7af30a31a42aea1dcfe8,00f130be2d96cb994c0b7465d6392b30675a8a1186688ef053e1a0c0b0be25dc,012bfafab8a84bdeb62a22af97ccafaec924cef3329680787dad16dbb2992ca1,0160c026ef59c9c0fe74cfb26082858ce4def298114ef904ad50b8d56a7f8b50,01718bfe393c5235cdb559eafe1bc3921238e063c087881fed2fe3db187c4b5e,0178c9d429db092f01bfd5c920be9b2f18791d3b69b4a12eaffdd6e59094bff0,...,fd8f418034de9007f6f636dbd961b3746aa360b2d801a281b803ed2d342e7df2,fdd3914d5fc6027178b777032931408226f90c373c2fec2333d1e1cc318d9f4d,fdf40e1207d4e8701228ae1e7e14e87b6ae86a6b3fcf1eaf69d9937cc090e0df,fe05d27ea5e730e08c296e22daf752c88e47bbc1d46e0094ac14ebc7ab290c93,fec3bcd6747e63495b5a1ea657d3a81f6381d9abe0f12bdff49f42a7c555f186,fece62762bff385172719d0f14c09bb6c5261aacd286d16f53a741aeba55e892,fedbd0d1998bec7e4353915662ac23aa8af1871096e5d250a4f40516f41e7901,fefcd51848ebcb12ab075eb031888e2c42171f488544c285ee8c5a2ed57fb9af,ff1e35e784727fe95f148d965c659251c5c8f5a2f134e2b06353e0a03b2806eb,fff280cabf75d279695720cda9f9d3c1c8df31bad3b209fb69bf31b6798712fd
